In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/duygu-analizi-icin-urun-yorumlari/magaza_yorumlari_duygu_analizi.csv


In [2]:
df=pd.read_csv("/kaggle/input/duygu-analizi-icin-urun-yorumlari/magaza_yorumlari_duygu_analizi.csv",encoding="utf-16")

In [3]:
df.head()

,Görüş,Durum
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",Olumsuz
1,hizli teslimat tesekkürler,Tarafsız
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,Olumsuz


In [4]:
df["Durum"].value_counts()

Durum
Olumlu      4253
Olumsuz     4238
Tarafsız    2938
Name: count, dtype: int64

In [5]:
import re
import nltk
from nltk.corpus import stopwords

# Temizleme

In [6]:
nltk.download("stopwords")
turkce_stopwords=stopwords.words("turkish")

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
turkce_stopwords

['acaba',
 'ama',
 'aslında',
 'az',
 'bazı',
 'belki',
 'biri',
 'birkaç',
 'birşey',
 'biz',
 'bu',
 'çok',
 'çünkü',
 'da',
 'daha',
 'de',
 'defa',
 'diye',
 'eğer',
 'en',
 'gibi',
 'hem',
 'hep',
 'hepsi',
 'her',
 'hiç',
 'için',
 'ile',
 'ise',
 'kez',
 'ki',
 'kim',
 'mı',
 'mu',
 'mü',
 'nasıl',
 'ne',
 'neden',
 'nerde',
 'nerede',
 'nereye',
 'niçin',
 'niye',
 'o',
 'sanki',
 'şey',
 'siz',
 'şu',
 'tüm',
 've',
 'veya',
 'ya',
 'yani']

In [8]:
def temizle(metin):
    if pd.isnull(metin):
        return ""

    metin=str(metin).lower()
    metin=re.sub(r'[^a-zçğıöşü\s]', '', metin)
    metin=' '.join([kelime for kelime in metin.split() if kelime not in turkce_stopwords])

    return metin

* Fonksiyona gelen metinlerin tüm harflerini küçülttük
* Harf dışındaki tüm karakterleri sildik
* Metinlerde geçen stopword'leri kaldırdık

-----------------------

In [9]:
df["temiz_yorum"]=df["Görüş"].apply(temizle)
df[["Görüş","temiz_yorum"]].head(10)

,Görüş,temiz_yorum
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",ses kalitesi ergonomisi rezalet sony olduğu al...
1,hizli teslimat tesekkürler,hizli teslimat tesekkürler
2,ses olayı süper....gece çalıştır sıkıntı yok.....,ses olayı süpergece çalıştır sıkıntı yokkablo ...
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,geldi bigün kullandık hemen bozoldu hiçtavsiye...
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,kulaklığın sesi kaliteli falan değil aleti öve...
5,Giriş seviyesindeki kullanıcılar için kabul ed...,giriş seviyesindeki kullanıcılar kabul edilebi...
6,kullanisli baya,kullanisli baya
7,Dezavantajlar : Pahalı ürün<br />Merhabalar bu...,dezavantajlar pahalı ürünbr merhabalar lens ba...
8,ÜRÜN GÜZEL BU PARALARA BAŞKA BULAMAZSINIZ,ürün güzel paralara başka bulamazsiniz
9,Tasarım ve kalite iyi olmasına rağmen yazma hı...,tasarım kalite iyi olmasına rağmen yazma hızı ...


* apply fonksiyonu her satırdaki yorumu alır, temizle() fonksiyonuna gönderir ve sonucu yeni sütuna yazar.

-------------------------------------

In [10]:
from sklearn.preprocessing import LabelEncoder

In [11]:
le=LabelEncoder()

df["Duygu"]=le.fit_transform(df["Durum"])
df[["Durum","Duygu"]].head()

,Durum,Duygu
0,Olumsuz,1
1,Tarafsız,2
2,Olumlu,0
3,Olumsuz,1
4,Olumsuz,1


"Durum" sütununu etiketlere ayırarak "Duygu" adında yeni sütuna yazdık

-----------------

# MODEL

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
X=df["temiz_yorum"]
y=df["Duygu"]

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

* Verinin %80'i eğitime, %20'si test için ayrıldı

--------------

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,classification_report

In [15]:
vektorizer=TfidfVectorizer(max_features=5000)
X_train_df=vektorizer.fit_transform(X_train)
X_test_df=vektorizer.transform(X_test)

* Makine öğrenmesi modelleri sayı ile çalıştığı için metinleri sayıya çevirmemiz lazım. Yukarda bu işlemi yaptık.
* Tf-idf ile metinde geçen kelimelerin önem derecesini belirledik
    * TF: Bir kelimenin yorum içinde kaç kez geçtiğini ifade eder.
    * IDF: Bu kelimenin tüm yorumlar arasında ne kadar yaygın olduğunu ifade eder.

----------------

In [16]:
model=MultinomialNB()

model.fit(X_train_df,y_train)

MultinomialNB()

* Bu model, kelime sayımlarına dayalı çalıştığı için TF-IDF ile oluşturulmuş vektörleri işleyebilir.

---------------

In [17]:
tahminler=model.predict(X_test_df)

In [18]:
print("Doğruluk: ",accuracy_score(y_test,tahminler))
print("\n Detaylı Rapor: \n",classification_report(y_test,tahminler,target_names=le.classes_))

Doğruluk:  0.6986001749781278

 Detaylı Rapor: 
               precision    recall  f1-score   support

      Olumlu       0.72      0.88      0.79       855
     Olumsuz       0.71      0.79      0.75       852
    Tarafsız       0.57      0.29      0.38       579

    accuracy                           0.70      2286
   macro avg       0.67      0.65      0.64      2286
weighted avg       0.68      0.70      0.67      2286



* Modelin doğruluğu %69-%70
* Model olumlu yorumları %72 doğrulukla tahmin etti ve %88’ini yakalayabildi.
* Model olumsuz yorumları %71 doğrulukla tahmin etti ve %79’unu yakalayabildi.
* Model tarafsız yorumları %57 doğrulukla tahmin etti ve %29'unu yakalayabildi. Model zorlanmış bu kısımda

------------------------------

In [19]:
def tahmin(metin):
    temiz_metin=temizle(metin)
    metin_tfidf=vektorizer.transform([temiz_metin])

    tahmin=model.predict(metin_tfidf)

    return tahmin[0]

In [20]:
yorum = "Ürün beklediğimden daha kaliteli çıktı, teşekkür ederim."

duygu_durumu = tahmin(yorum)

if duygu_durumu == 0:
    print("Metnin Duygu Durumu Olumlu")
elif duygu_durumu == 1:
    print("Metnin Duygu Durumu Olumsuz")
elif duygu_durumu == 2:
    print("Metnin Duygu Durumu Tarafsız")
else:
    print("Bir Durum Belirlenmedi")

Metnin Duygu Durumu Olumlu


Bir input alarak örnek tahmin yapıyoruz ve modelimiz doğru tahmin yapıyor.

**Not Defteri Hakkında Hem Oy Verip Hemde Yorum Yaparsanız Sevinirim.**